In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class ImageClassificationBase(nn.Module):
  def training_step(self, batch):
    images, labels = batch 
    out = self(images)                  # Generate predictions
    loss = F.cross_entropy(out, labels) # Calculate loss
    return loss
    
  def validation_step(self, batch):
    images, labels = batch 
    out = self(images)                    # Generate predictions
    loss = F.cross_entropy(out, labels)   # Calculate loss
    acc = accuracy(out, labels)           # Calculate accuracy
    return {'val_loss': loss.detach(), 'val_acc': acc}
        
  def validation_epoch_end(self, outputs):
    batch_losses = [x['val_loss'] for x in outputs]
    epoch_loss = torch.stack(batch_losses).mean()   # Combine losses
    batch_accs = [x['val_acc'] for x in outputs]
    epoch_acc = torch.stack(batch_accs).mean()      # Combine accuracies
    return {'val_loss': epoch_loss.item(), 'val_acc': epoch_acc.item()}
    
  def epoch_end(self, epoch, result):
    print("Epoch [{}], train_loss: {:.4f}, val_loss: {:.4f}, val_acc: {:.4f}".format(
            epoch, result['train_loss'], result['val_loss'], result['val_acc']))

In [2]:
class M3_Model(ImageClassificationBase):
  def __init__(self, in_channels = 1, out_channels = 32, kernel_size = 3, bias = False, stride = 1, padding = 0):
    super(M3_Model, self).__init__()
    self.network = nn.Sequential(
      nn.Conv2d(in_channels, out_channels, kernel_size, stride, padding, bias = False),
      nn.BatchNorm2d(out_channels),
      nn.ReLU(),
      nn.Conv2d(out_channels, 48, kernel_size,stride, padding, bias = False),
      nn.BatchNorm2d(48),
      nn.ReLU(),
      nn.Conv2d(48, 64, kernel_size, stride, padding, bias = False),
      nn.BatchNorm2d(64),
      nn.ReLU(),
      nn.Conv2d(64, 80, kernel_size, stride, padding, bias = False),
      nn.BatchNorm2d(80),
      nn.ReLU(),
      nn.Conv2d(80, 96, kernel_size, stride, padding, bias = False),
      nn.BatchNorm2d(96),
      nn.ReLU(),
      nn.Conv2d(96, 112, kernel_size, stride, padding, bias = False),
      nn.BatchNorm2d(112),
      nn.ReLU(),
      nn.Conv2d(112, 128, kernel_size, stride, padding, bias = False),
      nn.BatchNorm2d(128),
      nn.ReLU(),
      nn.Conv2d(128, 144, kernel_size, stride, padding, bias = False),
      nn.BatchNorm2d(144),
      nn.ReLU(),
      nn.Conv2d(144, 160, kernel_size, stride, padding, bias = False),
      nn.BatchNorm2d(160),
      nn.ReLU(),
      nn.Conv2d(160, 176, kernel_size, stride, padding, bias = False),
      nn.BatchNorm2d(176),
      nn.ReLU(),
    )
    self.fc = nn.Linear(11264, 10, bias = False)
    self.bn = nn.BatchNorm1d(10)
  
  def forward(self, x):
    x = self.network(x)
    x = x.view(x.size(0), -1)
    x = self.bn(self.fc(x))
    return x

def test():
  model = M3_Model()
  inputs = torch.randn((3,1,28,28))
  output = model(inputs)
  print(output.shape)
  print(output)

In [3]:
test()

torch.Size([3, 10])
tensor([[-0.4233, -1.0848, -1.4092, -0.8198,  0.7807, -1.1435, -1.0943,  1.0409,
          0.9850,  1.3827],
        [ 1.3789, -0.2432,  0.8069, -0.5881, -1.3328,  1.2922,  1.3224, -1.3494,
          0.3862, -0.9369],
        [-0.9555,  1.3280,  0.6023,  1.4078,  0.5521, -0.1488, -0.2281,  0.3084,
         -1.3713, -0.4458]], grad_fn=<NativeBatchNormBackward0>)


In [4]:
class M5_Model(ImageClassificationBase):
  def __init__(self, in_channels = 1, out_channels = 32, kernel_size = 5, stride = 1, padding  = 0, bias = False):
    super(M5_Model, self).__init__()
    self.network = nn.Sequential(
      nn.Conv2d(in_channels, out_channels, kernel_size, stride, padding, bias = False),
      nn.BatchNorm2d(out_channels),
      nn.ReLU(),
      nn.Conv2d(32, 64, kernel_size, stride, padding, bias = False),
      nn.BatchNorm2d(64),
      nn.ReLU(),
      nn.Conv2d(64, 96, kernel_size, stride, padding, bias = False),
      nn.BatchNorm2d(96),
      nn.ReLU(),
      nn.Conv2d(96, 128, kernel_size, stride, padding, bias = False),
      nn.BatchNorm2d(128),
      nn.ReLU(),
      nn.Conv2d(128, 160, kernel_size, stride, padding, bias = False),
      nn.BatchNorm2d(160),
      nn.ReLU(),
    )
    self.fc = nn.Linear(10240, 10, bias = False)
    self.bn = nn.BatchNorm1d(10)

  def forward(self, x):
    x = self.network(x)
    x = x.view(x.size(0), -1)
    x = self.fc(x)
    x = self.bn(x)
    return x
  
def test():
  model = M5_Model()
  inputs = torch.randn((3,1,28,28))
  output = model(inputs)
  print(output.shape)
  print(output)

In [5]:
test()

torch.Size([3, 10])
tensor([[ 1.3757,  1.2810, -1.0306,  0.0274,  0.1801,  1.3943, -0.6188,  1.1099,
         -1.4136, -1.2312],
        [-0.9705, -1.1593,  1.3535,  1.2107,  1.1245, -0.4933,  1.4106, -1.3131,
          0.7382,  0.0132],
        [-0.4051, -0.1217, -0.3229, -1.2381, -1.3046, -0.9010, -0.7918,  0.2032,
          0.6754,  1.2179]], grad_fn=<NativeBatchNormBackward0>)


In [7]:
class M7_Model(ImageClassificationBase):
  def __init__(self, in_channels = 1, out_channels = 48, kernel_size = 7, padding = 0, bias = False, stride = 1):
    super(M7_Model, self).__init__()
    self.network = nn.Sequential(
      nn.Conv2d(in_channels, out_channels, kernel_size,stride, padding, bias = False),
      nn.BatchNorm2d(out_channels),
      nn.ReLU(),
      nn.Conv2d(out_channels, 96, kernel_size,stride, padding, bias = False),
      nn.BatchNorm2d(96),
      nn.ReLU(),
      nn.Conv2d(96, 144, kernel_size,stride, padding, bias = False),
      nn.BatchNorm2d(144),
      nn.ReLU(),
      nn.Conv2d(144, 192, kernel_size, stride, padding, bias = False),
      nn.BatchNorm2d(192),
      nn.ReLU(),
    )

    self.fc = nn.Linear(192*4*4, 10, bias = False)
    self.bn = nn.BatchNorm1d(10)

  def forward(self, x):
    x = self.network(x)
    x = x.view(x.size(0), -1)
    x = self.bn(self.fc(x))
    return x

def test():
  model = M7_Model()
  inputs = torch.randn((3,1,28,28))
  output = model(inputs)
  print(output.shape)
  print(output)

In [8]:
test()

torch.Size([3, 10])
tensor([[ 1.4001, -1.3585,  1.2619,  1.3719, -0.5905, -0.8376,  0.5393, -1.3959,
          1.2628,  1.3034],
        [-0.8724,  1.0189, -0.0782, -0.9820,  1.4079,  1.4054, -1.4018,  0.5019,
         -0.0810, -0.1773],
        [-0.5277,  0.3396, -1.1837, -0.3899, -0.8174, -0.5677,  0.8625,  0.8940,
         -1.1818, -1.1262]], grad_fn=<NativeBatchNormBackward0>)


In [17]:
import torchvision
import torch
import torch.nn as nn
from torch.utils.data import Dataset
from torchvision import datasets
from torchvision import transforms
from torchvision.transforms import ToTensor
from torchvision.transforms import Resize
import matplotlib.pyplot as plt
import torch.nn.functional as F
import torch.optim as optim
from torchvision.transforms import ToTensor, Normalize, Compose
from torch.utils.data.dataloader import DataLoader

batch_size=128

In [18]:
train_data = datasets.MNIST(
    root="data",
    train=True,
    download=True,
    transform = ToTensor(),
)

test_data = datasets.MNIST(
    root="data",
    train=False,
    download=True,
    transform = ToTensor(),
)

train_dl = DataLoader(train_data, batch_size, shuffle=True)
val_dl = DataLoader(test_data, batch_size*2)

In [19]:
def accuracy(outputs, labels):
    _, preds = torch.max(outputs, dim=1)
    return torch.tensor(torch.sum(preds == labels).item() / len(preds))

In [20]:
@torch.no_grad()
def evaluate(model, val_loader):
    model.eval()
    outputs = [model.validation_step(batch) for batch in val_loader]
    return model.validation_epoch_end(outputs)

def fit(epochs, lr, model, train_loader, val_loader, opt_func=torch.optim.SGD):
    history = []
    optimizer = opt_func(model.parameters(), lr)
    for epoch in range(epochs):
        # Training Phase 
        model.train()
        train_losses = []
        for batch in train_loader:
            loss = model.training_step(batch)
            train_losses.append(loss)
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()
        # Validation phase
        result = evaluate(model, val_loader)
        result['train_loss'] = torch.stack(train_losses).mean().item()
        model.epoch_end(epoch, result)
        history.append(result)
    return history

num_epochs = 20
opt_func = torch.optim.Adam
lr = 0.001

In [21]:
model_1 = M3_Model()

In [ ]:
fit(num_epochs, lr, model_1, train_dl, val_dl, opt_func)

Epoch [0], train_loss: 0.3036, val_loss: 0.1683, val_acc: 0.9898


In [ ]:
model_2 = M5_Model()

In [ ]:
fit(num_epochs, lr, model_2, train_dl, val_dl, opt_func)

In [ ]:
model_3 = M7_Model()

In [ ]:
fit(num_epochs, lr, model_3, train_dl, val_dl, opt_func)